# Dependencies

In [13]:
#Dependencies
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import urllib.request
from bs4 import BeautifulSoup
print("Dependencies Loaded")
import datetime
from datetime import date
import pandas as pd

#Load Credentials
import sys
sys.path.insert(1, '/Users')
from master_config import sales_usr, sales_pwd, gmail_usr, gmail_pwd, gmail_usr2, gmail_pwd2, c_stud_usr, c_stud_pwd, sf_landing, work_email, calendly_link, app_link#from func_file import change_status_to, maj_element, stat_element

#Dependent Functions
#Refresh Soup; this is easier than typing out the full the BS4 function every time I want to refresh the soup
def newsoup():
    return BeautifulSoup(driver.page_source, 'html.parser')

#Salesforce lead profile pages are often made up of data tables; it's useful to be able to get these on impulse
def html_tables(soup):
    return soup.find_all('div', class_="bPageBlock brandSecondaryBrd secondaryPalette")

#Return list of rows in a table by calling its tablename
def return_rows(tablename):
    for table in html_tables(newsoup()):
        tblname = table.find('td',class_="pbTitle").get_text()
        if tablename in tblname:
            mytable = table
    rows = mytable.find('div',class_="pbBody").find_all('tr')
    return rows

#Send an email in gmail
def execute_email(emailsubject, emailbody):
    while str(newsoup().find('input',id=":oo")) != '<input aria-label="Subject" autocomplete="off" class="aoT" id=":oo" name="subjectbox" placeholder="Subject" spellcheck="true" tabindex="1"/>':
        None
    time.sleep(2)
    driver.find_element_by_class_name('aoT').send_keys(emailsubject)
    driver.find_element_by_id(':nk').send_keys(Keys.UP*15 + emailbody)
    time.sleep(1)
    driver.find_element_by_id(':oy').click()

#cls_button
def cls_button_locater():
    #get rows from the specified table
    rows = return_rows("Open Activities")
    #loop rows
    for row in rows:
        #the "Close Activity" Button is an <a></a> html tag; find the desired <a></a> tag.
        for a_tag in row.find_all('a'):
            if "Cls" in a_tag.get_text():
                #tag located: selenium navigates to the url/href contained therein
                driver.get(f"""{sf_landing}{str(a_tag).replace('<a class="actionLink" href="/','').split('"')[0]}""")

def get_TRX_and_recs():
    #get tally of outstanding Transcripts
    #get rows from the specified table
    rows = return_rows("Checklist Items (v2)")
    #initialize tally
    incompTRX = 0
    #loop rows
    for row in rows:
        #skip header row
        if row == rows[0]:
            #skip header row
            None
        else:
            #Outstanding TRX will match the below conditions
            if "College Transcript" in row.get_text() and "Completed" not in row.get_text():
                #add tally
                incompTRX = incompTRX + 1
            else:
                None
    #get tally of outstanding recommendations
    #get rows from the specified table
    rows = return_rows("Recommendations")
    #initialize tally
    incompREC = 0
    #loop rows
    for row in rows:
        if row == rows[0]:
            #skip header row
            None
        else:
            #Outstanding Recommendation letters will match the below conditions
            if "Incomplete" in row.get_text():
                #add tally
                incompREC = incompREC + 1
            else:
                None
    #determine email contents
    if incompTRX > 0:
        var1 = f"\n{incompTRX}x College Transcripts"
    else:
        var1 = ""
    #determine email contents
    if incompREC > 0:
        var2 = f"\n{incompREC}x Recommendation Letters"
    else:
        var2 = ""
    #final values to include in email
    incomps = [var1,var2]
    return incomps

def change_status_to(newvalue, element):
    if str(newvalue) != "nan":
        try:
            status = driver.find_element_by_id(f'{element}_ilecell')
            status.click()
            status.send_keys(Keys.RETURN)
            status.click()
            time.sleep(1)
            driver.find_element_by_id(element).send_keys(newvalue)
            status.click()
            driver.find_element_by_name('inlineEditSave').click()
            time.sleep(5)
            print("Operation Complete")
        except:
            print("Attempt failed; click manually")
    else:
        print('Skipping Status')


maj_element = "00N3600000LP34V"
stat_element = "00N0e000002cYq3"

Dependencies Loaded


# Initiate Driver & Login

In [14]:
driver = webdriver.Chrome('chromedriver')
print("Driver Initiated")

#Nagivate to Salesforce Login Url
driver.get(sf_landing);
userbox = driver.find_element_by_name('j_username')
userbox.send_keys(sales_usr)
passbox = driver.find_element_by_name('j_password')
passbox.send_keys(sales_pwd)
logbutton = driver.find_element_by_name('_eventId_proceed')
logbutton.click()

#Nagivate to Google Suite Login Url
driver.get('https://accounts.google.com/signin/v2');
userbox = driver.find_element_by_class_name('whsOnd.zHQkBf')
userbox.send_keys(gmail_usr)
nextbutton = driver.find_element_by_id('identifierNext')
ActionChains(driver).click(nextbutton).perform()
time.sleep(5)
passbox = driver.find_element_by_class_name('whsOnd.zHQkBf')
passbox.send_keys(gmail_pwd)
nextbutton = driver.find_element_by_id('passwordNext')
ActionChains(driver).click(nextbutton).perform()

time.sleep(2)

#Nagivate to Google Suite Login Url
driver.get('https://accounts.google.com/signin/v2/identifier?service=accountsettings&continue=https%3A%2F%2Fmyaccount.google.com%2F%3Futm_source%3Dsign_in_no_continue&ec=GAlAwAE&flowName=GlifWebSignIn&flowEntry=AddSession');
userbox = driver.find_element_by_class_name('whsOnd.zHQkBf')
userbox.send_keys(gmail_usr2)
nextbutton = driver.find_element_by_id('identifierNext')
ActionChains(driver).click(nextbutton).perform()
time.sleep(5)
passbox = driver.find_element_by_name('password')
passbox.send_keys(gmail_pwd2)
nextbutton = driver.find_element_by_id('passwordNext')
ActionChains(driver).click(nextbutton).perform()

#Navigate to Cardinal Students Login Url and Load Certs
driver.get("https://csprd.cua.edu/psp/csprd/EMPLOYEE/SA/?cmd=logout")
time.sleep(2)
driver.find_element_by_id('userid').send_keys(c_stud_usr)
driver.find_element_by_id('pwd').send_keys(c_stud_pwd)
driver.find_element_by_class_name('ps-button').click()


time.sleep(2)

#go to gmail home
driver.switch_to.window(driver.window_handles[0])
driver.get('https://mail.google.com/mail/u/0/#inbox')

driver.get(sf_landing);
print("Operation Complete")

Driver Initiated
Operation Complete


# Login to Social Media and Jobs

In [31]:
driver.switch_to.window(driver.window_handles[0])
from master_config import glass_door_usr, glass_door_pwd, linkedin_usr, linkedin_pwd, indeed_usr, indeed_pwd, fbchat_usr, fbchat_pwd, patreon_usr, patreon_pwd, ao3_usr, ao3_pwd

#Glassdoor
driver.get('https://www.glassdoor.com/profile/login_input.htm?userOriginHook=HEADER_SIGNIN_LINK')
time.sleep(1)
driver.find_element_by_id("userEmail").send_keys(glass_door_usr)
time.sleep(1)
driver.find_element_by_id("userPassword").send_keys(glass_door_pwd)
driver.find_element_by_class_name("gd-ui-button.minWidthBtn.css-8i7bc2").click()
time.sleep(1)

#LinkedIn
driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')
time.sleep(1)
driver.find_element_by_id("username").send_keys(linkedin_usr)
time.sleep(1)
driver.find_element_by_id("password").send_keys(linkedin_pwd)
driver.find_element_by_class_name("btn__primary--large.from__button--floating").click()
time.sleep(1)

#Indeed
driver.get('https://secure.indeed.com/account/login?hl=en_US&co=US&continue=https%3A%2F%2Fwww.indeed.com%2F&tmpl=desktop&service=my&from=gnav-util-homepage&jsContinue=https%3A%2F%2Fwww.indeed.com%2F&empContinue=https%3A%2F%2Faccount.indeed.com%2Fmyaccess&_ga=2.52749928.1724983646.1617887947-312372538.1617887947')
time.sleep(1)
driver.find_element_by_id("login-email-input").send_keys(indeed_usr)
time.sleep(1)
driver.find_element_by_id("login-password-input").send_keys(indeed_pwd)
driver.find_element_by_id("login-submit-button").click()
time.sleep(1)

#Facebook Chat
driver.get('https://www.messenger.com/login/')
time.sleep(5)
driver.find_element_by_id("email").send_keys(fbchat_usr)
time.sleep(1)
driver.find_element_by_id("pass").send_keys(fbchat_pwd)
driver.find_element_by_id("loginbutton").click()
time.sleep(1)

#AO3 Login
driver.get("https://archiveofourown.org/")
time.sleep(4)
driver.find_element_by_id("tos_agree").click()
driver.find_element_by_id("accept_tos").click()
time.sleep(3)
driver.find_element_by_id("login-dropdown").click()
time.sleep(1)
driver.find_element_by_id("user_session_login_small").send_keys(ao3_usr)
driver.find_element_by_id("user_session_password_small").send_keys(ao3_pwd)
driver.find_element_by_id("user_remember_me_small").click()
driver.find_element_by_id("user_session_login_small").send_keys(Keys.RETURN)
time.sleep(1)

#Patreon Login
driver.get('https://www.patreon.com/login')
time.sleep(2)
driver.find_element_by_id("email").send_keys(patreon_usr)
time.sleep(1)
driver.find_element_by_id("password").send_keys(patreon_pwd)
driver.find_element_by_class_name("sc-fKFyDc.dgckCZ").click()
time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="login-email-input"]"}
  (Session info: chrome=90.0.4430.93)


In [ ]:
# import necessary libraries
import os
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)

#################################################
# Flask Setup
#################################################
app = Flask(__name__)

#################################################
# Database Setup
#################################################

from flask_sqlalchemy import SQLAlchemy
# app.config['SQLALCHEMY_DATABASE_URI'] = os.environ.get('DATABASE_URL', '') or "sqlite:///db.sqlite"
# app.config['SQLALCHEMY_DATABASE_URI'] = os.environ.get('DATABASE_URL')

app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite:///db.sqlite"

db = SQLAlchemy(app)

from .models import Pet

# create route that renders index.html template
@app.route("/")
def home():
    return render_template("index.html")


# Query the database and send the jsonified results
@app.route("/send", methods=["GET", "POST"])
def send():
    if request.method == "POST":
        name = request.form["petName"]
        lat = request.form["petLat"]
        lon = request.form["petLon"]

        pet = Pet(name=name, lat=lat, lon=lon)
        db.session.add(pet)
        db.session.commit()
        return redirect("/", code=302)

    return render_template("form.html")


@app.route("/api/pals")
def pals():
    results = db.session.query(Pet.name, Pet.lat, Pet.lon).all()

    hover_text = [result[0] for result in results]
    lat = [result[1] for result in results]
    lon = [result[2] for result in results]

    pet_data = [{
        "type": "scattergeo",
        "locationmode": "USA-states",
        "lat": lat,
        "lon": lon,
        "text": hover_text,
        "hoverinfo": "text",
        "marker": {
            "size": 50,
            "line": {
                "color": "rgb(8,8,8)",
                "width": 1
            },
        }
    }]

    return jsonify(pet_data)


if __name__ == "__main__":
    app.run()

# Redo Login (If Salesforce AutoLogs Out)

In [ ]:
driver.switch_to.window(driver.window_handles[0])
from config import glass_door_usr, glass_door_pwd, linkedin_usr, linkedin_pwd, indeed_usr, indeed_pwd, fbchat_usr, fbchat_pwd, patreon_usr, patreon_pwd, ao3_usr, ao3_pwd

#Glassdoor
driver.get('https://www.glassdoor.com/profile/login_input.htm?userOriginHook=HEADER_SIGNIN_LINK')
time.sleep(1)
driver.find_element_by_id("userEmail").send_keys(glass_door_usr)
time.sleep(1)
driver.find_element_by_id("userPassword").send_keys(glass_door_pwd)
driver.find_element_by_class_name("gd-ui-button.minWidthBtn.css-8i7bc2").click()
time.sleep(1)

#LinkedIn
driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')
time.sleep(1)
driver.find_element_by_id("username").send_keys(linkedin_usr)
time.sleep(1)
driver.find_element_by_id("password").send_keys(linkedin_pwd)
driver.find_element_by_class_name("btn__primary--large.from__button--floating").click()
time.sleep(1)

#Indeed
driver.get('https://secure.indeed.com/account/login?hl=en_US&co=US&continue=https%3A%2F%2Fwww.indeed.com%2F&tmpl=desktop&service=my&from=gnav-util-homepage&jsContinue=https%3A%2F%2Fwww.indeed.com%2F&empContinue=https%3A%2F%2Faccount.indeed.com%2Fmyaccess&_ga=2.52749928.1724983646.1617887947-312372538.1617887947')
time.sleep(1)
driver.find_element_by_id("login-email-input").send_keys(indeed_usr)
time.sleep(1)
driver.find_element_by_id("login-password-input").send_keys(indeed_pwd)
driver.find_element_by_id("login-submit-button").click()
time.sleep(1)

#Facebook Chat
driver.get('https://www.messenger.com/login/')
time.sleep(5)
driver.find_element_by_id("email").send_keys(fbchat_usr)
time.sleep(1)
driver.find_element_by_id("pass").send_keys(fbchat_pwd)
driver.find_element_by_id("loginbutton").click()
time.sleep(1)

#AO3 Login
driver.get("https://archiveofourown.org/")
time.sleep(4)
driver.find_element_by_id("tos_agree").click()
driver.find_element_by_id("accept_tos").click()
time.sleep(3)
driver.find_element_by_id("login-dropdown").click()
time.sleep(1)
driver.find_element_by_id("user_session_login_small").send_keys(ao3_usr)
driver.find_element_by_id("user_session_password_small").send_keys(ao3_pwd)
driver.find_element_by_id("user_remember_me_small").click()
driver.find_element_by_id("user_session_login_small").send_keys(Keys.RETURN)
time.sleep(1)

#Patreon Login
driver.get('https://www.patreon.com/login')
time.sleep(2)
driver.find_element_by_id("email").send_keys(patreon_usr)
time.sleep(1)
driver.find_element_by_id("password").send_keys(patreon_pwd)
driver.find_element_by_class_name("sc-fKFyDc.dgckCZ").click()
time.sleep(1)

In [76]:
driver.switch_to.window(driver.window_handles[0])
driver.get(sf_landing);
userbox = driver.find_element_by_name('j_username')
userbox.send_keys(sales_usr)
passbox = driver.find_element_by_name('j_password')
passbox.send_keys(sales_pwd)
logbutton = driver.find_element_by_name('_eventId_proceed')
logbutton.click()

# Data Gathering Module

In [85]:
tag = "JC - "
driver.switch_to.window(driver.window_handles[0])
all_fwups = driver.find_elements_by_partial_link_text(f'{tag}')
for fwup in all_fwups[82:]:
#    ActionChains(driver).key_down(Keys.COMMAND).click(fwup).key_up(Keys.COMMAND).perform()
    None
print('moving')
dataset = []
for x in driver.window_handles:
    try:
        driver.switch_to.window(driver.window_handles[1])
        soup = newsoup()
        taskdetail = soup.find('h2', class_="mainTitle").get_text()
        if taskdetail == "Task Detail":            
            duedate = soup.find("div", id="tsk4_ileinner").get_text()
            fwup = soup.find("div", id="tsk5_ileinner").get_text()
            if soup.find('div', id="tsk12_ileinner").get_text() == "Completed":
                duedate = "COMPLETED TASK - DNC"
                fwup = "COMPLETED TASK - DNC"
            driver.find_element_by_xpath("/html/body/div/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/a").click()
            soup = newsoup()
        else:
            duedate = "app audit"
            fwup = ""
        name = soup.find("div", id="con2_ileinner").get_text()
        phone = soup.find("div", id="con12_ileinner").get_text()
        initial_status = soup.find("div", id="00N0e000002cYq3_ileinner").get_text()
        if phone == "\xa0":
            phone = soup.find("div", id="con13_ileinner").get_text()
        major = soup.find("div", id="00N3600000LP34V_ileinner").get_text()
        createdby = soup.find('div', id='CreatedBy_ileinner').get_text().split(" ")
        initial_date = createdby[len(createdby) - 3]
        url = driver.current_url
        email = soup.find('div', id='con15_ileinner').get_text()
        #populate dictionary with the mined data
        fwupdict = {
            "fwup":fwup.replace("JC - ",''),
            "name":name,
            "major":major,
            "duedate":duedate,
            "initial date":initial_date,
            "initial status":initial_status,
            "phone":phone.replace("+1 ",""),
            "email":"",
            "emailfwup":"",
            "calltext":"",
            "comments":"",
            "newtask":"",
            "date":"",
            "time":"",
            "status":"",
            "url":url,
            "leademail":email
        }
        dataset.append(fwupdict)
        driver.close()
    except:
        print('finished')
        break
driver.switch_to.window(driver.window_handles[0])
df = pd.DataFrame(dataset)
csvname = f"fwups{datetime.datetime.now()}".split(':')[0]
df.to_csv(f'{csvname}.csv', index=False)

moving
finished


# Update DataBase

In [30]:
import datetime
driver.switch_to.window(driver.window_handles[0])
#initialize header and body lists
header_list = []
body_list = []

#get URL: first list
driver.get(f'{sf_landing}003?fcf=00B0e0000087I1d')

#GET HEADER
soup = newsoup()
table = soup.find('div', class_="listBody")
header = table.find('div', class_="x-grid3-header")

for td in header.find_all('td')[3:]:
    header_list.append(td.get_text())
header_list.append('Href')

#GET ROWS
myswitch = True
while myswitch == True:    
    try:
        soup = newsoup()
        table = soup.find('div', class_="listBody")
        body = table.find('div', class_="x-grid3-body")
        for tr in body.find_all('tr'):
            tds = tr.find_all('td')
            mydict = {}
            mydate = tds[3].get_text().split('/')
            newdate = datetime.date(int(mydate[2]), int(mydate[0]), int(mydate[1]))
            d_entry = {header_list[0]:newdate}
            mydict.update(d_entry)
            for x in range(8):
                d_entry = {header_list[x+1]:tds[x+4].get_text()}
                mydict.update(d_entry)
            mydict.update({header_list[9]:tds[1].get_text()})
            body_list.append(mydict)
        driver.find_element_by_partial_link_text('Next').click()
        time.sleep(5)
    except:
        myswitch = False

#get URL: second list
driver.get(f"{sf_landing}00Q?fcf=00B0e0000087I1n")

#GET ROWS
myswitch = True
while myswitch == True:    
    try:
        soup = newsoup()
        table = soup.find('div', class_="listBody")
        body = table.find('div', class_="x-grid3-body")
        for tr in body.find_all('tr'):
            tds = tr.find_all('td')
            mydict = {}
            mydate = tds[3].get_text().split('/')
            newdate = datetime.date(int(mydate[2]), int(mydate[0]), int(mydate[1]))
            d_entry = {header_list[0]:newdate}
            mydict.update(d_entry)
            for x in range(8):
                d_entry = {header_list[x+1]:tds[x+4].get_text()}
                mydict.update(d_entry)
            mydict.update({header_list[9]:tds[1].get_text()})
            body_list.append(mydict)
        driver.find_element_by_partial_link_text('Next').click()
        time.sleep(5)
    except:
        myswitch = False

        
#TRANSFORM DATA
df = pd.DataFrame(body_list)
mydates = df['Created Date'].to_list()
newdates = []
for mydate in mydates:
    sd = str(mydate).split('-')
    newdate = f'{sd[0]}-{sd[1]}'
    newdates.append(newdate)
srclst = df['Lead Source'].to_list()
newsrclst = []
x = "MSPS"
y = "LinkedIn"
z = "Other"
for src in srclst:    
    try:
        if x in src:
            newsrc = f'{x} Lead'
            newsrclst.append(newsrc)
        elif y in src:
            newsrc = f'{y} Lead'
            newsrclst.append(newsrc)
        else:
            newsrc = z
            newsrclst.append(newsrc)
    except:
        newsrc = z
        newsrclst.append(newsrc)
newmajors = []
for x in range(len(df["Created Date"])):
    if str(df["Academic Plan"][x]).lower() != '\xa0':
        newmajors.append(df["Academic Plan"][x])
    else:
        newmajors.append(df["Anticipated Major"][x])
newdf = pd.DataFrame({"Created Date":df["Created Date"],
                    "Created Month":newdates,
                    "Anticipated Major":newmajors,
                    "Lead Status":df["Lead Status"],
                    "Name":df["Name"],
                    "Email":df["Email"],
                    "Source":df["Source"],
                    "Lead Source":df["Lead Source"],
                    "Processed Lead Source":newsrclst,
                     "Href":df["Href"]})
print(len(newdf["Created Date"]))
for x in range(len(newdf["Created Date"])):
    if "test" in newdf["Name"][x].lower():
        newdf = newdf.drop([x])
    try:
        email = str(newdf["Email"][x])
        if "@cua.edu" in email:
            if email[0] not in "0123456789":
                newdf = newdf.drop([x])
    except:
        email = "----------------\nERROR AT THIS INDEX\n----------------"
print(len(newdf["Created Date"]))
newdf.to_csv(f'DA-Data-Processed.csv', index=False)

1544
1488


# Autoloop

In [77]:
csvname = "fwups2021-05-10 17"
print(csvname)

fwups2021-05-10 17


In [78]:
driver.switch_to.window(driver.window_handles[0])
newdf = pd.read_csv(f"{csvname}.csv")
n = 0
driver.switch_to.window(driver.window_handles[0])
#establish variable to allow for only 3 options; prevent errors
errorprevention = 0
#ask user if they're doing followups or new leads
#New Note Module
x = -1
for url in newdf['url'].tolist():
    driver.get(url)
    x = x + 1
    #make soup
    soup = newsoup()
    #### EEEEEEE  M     M    A   II  LL
    #### EE       MM   MM   A A  II  LL
    #### EEEEEEE  M M M M  AAAAA II  LL
    #### EE       M  M  M  A   A II  LL
    #### EEEEEEE  M     M  A   A II  LLLLLLL
    email_type = newdf['email'].tolist()[x]#input("Press 'Enter' to send email (or 'n' to skip): ")
    email = newdf['leademail'].tolist()[x]
    recipient = soup.find("div", id="con2_ileinner").get_text()
    gender = soup.find('div', id="00N3600000LP34w_ileinner").get_text()
    if gender == "Male":
        honorific = "Mr. "
    elif gender == "Female":
        honorific = "Ms. "
    else:
        honorific = ""
    subjectinput = newdf['major'].tolist()[x]
    subjectoutput = ""
    if subjectinput == "Data Analytics (M.S.)":
        subjectoutput = "Master of Science in Data Analytics"
    elif subjectinput == "Data Analytics - Cert":
        subjectoutput = "Certificate in Data Analytics"
    elif subjectinput == "Cyber Security & Privacy - Cert":
        subjectoutput = "Certificate in Cyber Security"
    else:
        print('!!!!!!!!!!!!! - Invalid Entry - Retry - !!!!!!!!!!!!!')
        subjectoutput = input("Fix error: ")
#    if email_type != "n":
#        driver.get(f'{work_email}{email}')
    followup = ""
    if email_type == "y":
        followup = newdf['emailfwup'].tolist()[x]
        if followup == "y":
            fwupsnippet = f'I just wanted to reach out to you one more time regarding your inquiry into our {subjectoutput}. '
        elif followup == "c":
            fwupsnippet = f"I notice you previously asked for information on the {subjectoutput}. "
        else:
            fwupsnippet = ""
        driver.get(f'{work_email}{email}')
        emailsubject = f'Earn a {subjectoutput} at The Catholic University of America'
        body_text = f"""Thank you for your interest in The Catholic University of America!\n\n          My name is John. {fwupsnippet}My role is to offer any help you may need to gather info and start an application.\n\n***Call us for a consultation to redeem your FREE application fee waiver!***\nPick any time between 11AM and 5PM Monday-Friday that works for you: {calendly_link}\n\n          To start an application, click here: {app_link}"""
        emailbody = f"{honorific}{recipient}\n\n{body_text}"
        execute_email(emailsubject, emailbody)
    elif email_type == "a":
        stoperror = driver.current_url
        soup = newsoup()
        rows = return_rows("Applications")
        for row in rows:
            if row == rows[0]:
                None
            elif "Incomplete" not in row.get_text():
                None
            else:
                aud_app = row.find_all('a')[1].get_text()
        driver.find_element_by_partial_link_text(aud_app).click()
        while driver.current_url == stoperror:
            None
        time.sleep(2)
        soup = newsoup()
        t_r = get_TRX_and_recs()
        driver.get(f'{work_email}{email}')
        audit_text = f"""          This is John from CUA's {subjectoutput} program.\n          I just wanted to reach out regarding your application. It looks like we still need these remaining items:{t_r[0]}{t_r[1]}\n\n         If you (or your recommenders) have already submitted the documents described above, please let me know. This may be the result of a technical error.\n         Thanks very much for your interest in the program! Keep up the good work!"""
        emailsubject = f"Update on Your Application to CUA's {subjectoutput}"
        emailbody = f"{honorific}{recipient}\n\n{audit_text}"
        execute_email(emailsubject, emailbody)        
    elif email_type == "s":
        subjectinput = soup.find("div", id="00N3600000LP34V_ileinner").get_text()
        subjectoutput = ""
        driver.get(f'{work_email}{email}')
        errorprevention = 0
        while errorprevention < 1:
            if subjectinput == "Data Analytics (M.S.)":
                subjectoutput = "Master of Science in Data Analytics"
                sought_program = "Degree Seeking"
                sought_school = "School of Engineering"
                sought_deg = "Data Analytics (M.S.)"
                errorprevention = errorprevention + 1
            elif subjectinput == "Data Analytics - Cert":
                subjectoutput = "Certificate in Data Analytics"
                sought_program = "Certificate"
                sought_school = "School of Engineering Non-Degree"
                sought_deg = "Data Analytics - Cert"        
                errorprevention = errorprevention + 1
            elif subjectinput == "Cyber Security & Privacy - Cert":
                subjectoutput = "Certificate in Cyber Security"
                sought_program = "Certificate"
                sought_school = "School of Engineering Non-Degree"
                sought_deg = "Cyber Security & Privacy - Cert"        
                errorprevention = errorprevention + 1
            else:
                errorprevention = errorprevention + 1
                subjectoutput = "Master of Science in Data Analytics"
                sought_program = "Degree Seeking"
                sought_school = "School of Engineering"
                sought_deg = "Data Analytics (M.S.)"
                errorprevention = errorprevention + 1
        #Greeting
        driver.get(f'{work_email}{email}')
        emailsubject = "Data Analytics at CUA: Info You Requested"
        greet_text = "          Thanks so much for speaking with me today.\n\n          I'll be sending you 3 emails in just a moment more in-depth info on the program: specifically - \n\n(1)How to Apply\n(2)Class Overview/Brochure\n(3)Other Offices/Resources at CUA\n\n          Thanks again for speaking with me. I think you'd make a great candidate and I look forward to hearing from you again soon!"
        emailbody = f"{honorific}{recipient}\n\n{greet_text}"
        #execute_email(emailsubject, emailbody)
        #Application Link
        driver.get(f'{work_email}{email}')
        emailsubject = f'Applying to Earn a {subjectoutput} at The Catholic University of America'
        app_text = f"          It was a pleasure speaking with you today. Based on our conversation, I think you're a good fit for our Master of Science in Data Analytics Degree program.\n          To begin your application, follow this link: {app_link}  \n          You'll need to create a login. Once you've logged in, you'll want to make sure to pick the following options from the drop-down menus.\n-{sought_program}\n-{sought_school}\n-{sought_deg}\n-[Desired Semester] (i.e. Summer/Fall 2021)\n          If you have any trouble completing the application, please call me at my office# (included below). I can walk you through the process.\n         Thanks very much for speaking with me. I look forward to hearing from you soon!"
        emailbody = f"""{honorific}{recipient}\n\n{app_text}
        """
        execute_email(emailsubject, emailbody)
        #Brochure
        driver.get(f'{work_email}{email}')
        emailsubject = f'Data Analytics at CUA: Email Brochure'
        broch_text = "          To see the presentation we use for our weekly webinar on the program, visit: https://docs.google.com/presentation/d/1CxI03mMTERenu_sI3a6DP3uUH8F4PQkKbdewTSVBTnA/edit?usp=sharing\n\nVisit https://youtu.be/3tKYwsfXIUc to watch a recording of our weekly webinar covering the DA program.\n\nSee a list of available classes below. For more classes and a comprehensive overview, visit: https://metro.catholic.edu/academics/graduate/ms-data-analytics/index.html\n\nCore Classes (First 4/10 classes in the degree)\n-----------------------------------------------------------------\nDA 501: Introduction to Data Science and Python (3 credit hours)\nA hands-on introduction to the field of Data Science and its applications. Covers a wide range of topics to provide an overview of the use of data in different fields. Provides hands-on practice with basic tools and methods of data analysis. Prepares students to use data in their field of study and in their work and to effectively communicate quantitative findings. Focus is on the use of Python in data analysis and mastering tools for acquiring, parsing, manipulating, and preparing data for statistical analysis.\nDA 514: Applied Statistics and Data Analysis (3 credit hours)\nThis is a second course in statistics. The course will focus on Design of experiments and Analysis of Variance, Categorical data analysis, Regression analysis, Time series/forecasting, and Data visualization.\n\nDA 515: Introduction to Machine Learning (3 credit hours)\nThe course introduces principles, algorithms, and applications of machine learning from the point of view of modeling and prediction; formulation of learning problems; representation, over-fitting, generalization; clustering, classification, probabilistic modeling; and methods such as support vector machines, hidden Markov models, and Bayesian networks.\n\nDA 516: Applications of Data Analytics and Development (3 credit hours)\nThe focus will be on programming and data manipulation techniques for constructing analytics-based applications.  Topics include SQL or no-SQL databases, using web service API’s to acquire data, introduction to Hadoop and MapReduce, and use of third-party analytic component API’s.\n-----------------------------------------------------------------\nMSDA Electives (Ask advisors about scheduling for electives in advance; electives are offered on a rotating basis; visit https://metro.catholic.edu/academics/graduate/ms-data-analytics/degree-requirement.html for a full list of electives\n-----------------------------------------------------------------\nCSC 447 - Artificial Intelligence (3 Credit Hours)\nTopics may include state space search, heuristic search, knowledge representation techniques, expert systems, automated reasoning, definitions of intelligence, computer problem solving, game playing, pattern recognition, theorem proving, semantic information, processing, evolutionary systems, and heuristic programming. Prerequisite: Instructor's permission.\n\nCSC 410 - Fundamentals of Cloud Computing (3 Credit Hours)\nOverview of the field of Cloud Computing, its enabling technologies, main building blocks, and hands-on experience through projects utilizing public cloud infrastructures, such as Amazon Web Services (AWS) and Microsoft Azure. Cloud computing services are being adopted widely across a variety of organizations in many domains. Simply, cloud computing is the delivery of computing as a service over a network, whereby distributed resources are rented, rather than owned, by an end user as a utility. Prerequisites: CSC 363 and CSC 323\n\nCSC 728 - Visualization (3 Credit Hours)\nThe course focuses on visualization of scientific data. Both visualization principles and practical design issues are addressed. The course introduces the visualization pipeline. It covers the visualization of scalar data, vector data, and tensor data. It also covers image visualization, volume visualization and finally information visualization. It discusses the effective use of visualization in various areas of the natural sciences, and examples of application will be drawn from these areas. It emphasizes the importance of visualization in understanding observations, examining theories, and fostering new scientific hypothesis. representation and presentation, document visualization, and dynamic exploration Permission of instructor.\n\nCSC 651 - Multimedia Processing and Information Retrieval (3 Credit Hours)\nThis course covers topics including multimedia systems, multimedia applications, image compression and processing, video compression and processing, content-based image retrieval, and content-based video indexing and retrieval. Prerequisite: EE 634 or Permission of instructor.\n\nCSC 641 - Data Mining (3 Credit Hours)\nIntroduction to data mining techniques, including data preprocessing, data mining primitives, association rules, decision trees, cluster analysis, classification and machine learning, data visualization, and data warehousing. Applications from a wide variety of domains will be studied. Prerequisite: CSC 541 or Permission of instructor."
        emailbody = f"{honorific}{recipient}\n\n{broch_text}"
        execute_email(emailsubject, emailbody)
        #Resources
        driver.get(f'{work_email}{email}')
        emailsubject = f'Resources at CUA'
        resource_text = "          Included in this email are some important CUA resources for you to take advantage of.\n\nCAREER SERVICES:\nhttps://success.catholic.edu/about-us/contact-us/index.html\nPhone: 202-319-6262\nEmail: success@cua.edu\nENROLLMENT SERVICES:\nhttps://enrollment-services.catholic.edu/contact-us/index.html\nPhone: 202-319-5300\nEmail: cua-enrollmentservices@cua.edu\n\nFINANCIAL AID SERVICES:\nhttps://financial-aid.catholic.edu/contact-us/index.html\nPhone: 202-319-5307\nEmail: cua-finaid@cua.edu"
        emailbody = f"{honorific}{recipient}\n\n{resource_text}"
        execute_email(emailsubject, emailbody)
        if newdf['emailfwup'].tolist()[x] == "j":
            driver.get(f'{work_email}{email}')
            emailsubject = f'DA Program Graduate Job Placement'
            jobs_text = f"            I have an applicant, {honorific}{recipient}, who'd like to learn about job placement, potentially from yourself and/or past graduates of the program.\n            Would you be able to assist?"
            emailbody = f"Dr. Chang\n\n{jobs_text}"
            while str(newsoup().find('input',id=":oo")) != '<input aria-label="Subject" autocomplete="off" class="aoT" id=":oo" name="subjectbox" placeholder="Subject" spellcheck="true" tabindex="1"/>':
                None
            time.sleep(2)
            driver.find_element_by_id(':od').click()
            driver.find_element_by_id(':o6').send_keys("changl@cua.edu\n")
            execute_email(emailsubject, emailbody)        
    elif email_type == "d":
        #APPLICATIONS SECTION
        for td_tag in range(len(html_tables)):
            myvar = html_tables[td_tag].find('td', class_='pbTitle').get_text()
            #find the "Applications" table and save it's contents to the apps_table variable
            if myvar == "Applications":
                apps_table = html_tables[td_tag]
        tdata = apps_table.find_all('tr')[2].find_all('td')
        app_semester = tdata[3].get_text() +" "+ tdata[4].get_text()
        driver.get(f'{work_email}{email}')
        emailsubject = f"Your Application to Catholic U's {subjectoutput}"
        deferral_text = f"          This is John, from Catholic University of America. I hope you've been well!\n\n          We worked together on your {app_semester} application for the {subjectoutput} and you had expressed interested in deferring to Spring 2021 and there are still open seats left!\n\n          Please let me know if you'd like to enroll and I may be able to change your old app to Spring 2021. The deadline for submission is December 17th, 2020.\n          Click here to start a new app: https://cardinaladmissions.force.com/\n          Click here to schedule a phone appointment with me: {calendly_link}\n\n          Thanks so much for your interest in our programs! I hope to speak to you soon."
        emailbody = f"{honorific}{recipient}\n\n{deferral_text}"
        execute_email(emailsubject, emailbody)
    elif email_type == "q":
        driver.get(f'{work_email}{email}')
        emailsubject = f"CUA's {subjectoutput}: Check-In"
        qual_text = f"          Hi there! This is John from CUA, just checking in about your interest in the {subjectoutput}. Based on our earlier conversation, I think you'd make a great candidate.\n\n          If you'd like to start an application, you can apply here: {app_link}\n\n          Make sure to email me when you've begun, and I'll approve your $60 fee waiver.\n          And if you need to meet/speak with me directly, feel free to call my phone number (202-695-2906) or make an appointment with me here: {calendly_link}\n          Thanks again for your interest in the program, and please feel free to reach out to me with any other questions you may have."
        emailbody = f"{honorific}{recipient}\n\n{qual_text}"
        execute_email(emailsubject, emailbody)
    elif email_type == "n":
        print("Skipping Email")
        emailbody = "(no email sent for this note)"
    else:
        print("Invalid Key. Try Again.")
    print("Emails finished")

    #### NN   NN  OOOOOOO  TTTTTT EEEEEE
    #### NNN  NN  OO   OO    TT   EE
    #### NN N NN  OO   OO    TT   EEEEEE
    #### NN  NNN  OO   OO    TT   EE
    #### NN   NN  OOOOOOO    TT   EEEEEE
    driver.get(url);
    time.sleep(4)
    #find subjectline and commentsline strings from the df
    subject = newdf['calltext'].tolist()[x].replace("cle;",'called; lvm; emailed')
    comments = newdf['comments'].to_list()[x]
    #determine the appropriate button to click
    #execute click
    soup = newsoup()
    if str(newdf['fwup'].tolist()[x]) != 'nan': 
        cls_button_locater()
    else:
        while driver.current_url == url:
            try:
                driver.get(url)
                driver.find_element_by_name('new').click()
                time.sleep(1)
            except:
                print('success')
    #execute note
    if followup == "c":
        c = "campaign: "
    else:
        c = ""
    driver.find_element_by_name('tsk5').clear()
    driver.find_element_by_name('tsk5').send_keys(f'JC - {c}{subject}')
    driver.find_element_by_name('tsk6').send_keys(f'{comments}\n Email:\n{emailbody}')
    driver.find_element_by_name('save').click()            
    print("Operation Complete")
    driver.get(url)
    #New Task Module
    if str(newdf['newtask'].tolist()[x]) != 'nan':
        preventerror_url = driver.current_url
        #INPUTFIELD
        targetdate = str(newdf['date'].tolist()[x]).replace('"','')#input("What date?(mm/dd): ")
        targettime = str(newdf['time'].tolist()[x])#input('What time?(i.e. "8" or "4"): ')
        subject = newdf['newtask'].tolist()[x]#input("Subject: ")
        #driver.switch_to.window(driver.window_handles[n])
        driver.find_element_by_name('task').click()
        while url == driver.current_url:
            preventerror_url = driver.current_url
            try:
                driver.find_element_by_name('task').click()
            except:
                print('still trying')
        print("Task Started")
        driver.find_element_by_name('tsk5').clear()
        driver.find_element_by_name('tsk5').send_keys(f'JC - {subject}')
        print("typed subject")
        driver.find_element_by_name('tsk4').send_keys(f'{targetdate}')
        print("typed date")
        driver.find_element_by_name('reminder_dt_time').send_keys(targettime)
        print("entered time")
        driver.find_elements_by_name('save')[1].click()
        print("Saved")
        print("Operation Complete")
    else:
        print("Skipping New Task")
    #Status Module
    newvalue = newdf['status'].tolist()[x]
    change_status_to(newvalue, stat_element)

Emails finished
Operation Complete
Task Started
typed subject
typed date
entered time
Saved
Operation Complete
Operation Complete
Emails finished
Operation Complete
Task Started
typed subject
typed date
entered time
Saved
Operation Complete
Operation Complete
Emails finished
Operation Complete
Task Started
typed subject
typed date
entered time
Saved
Operation Complete
Operation Complete
Emails finished
Operation Complete
Task Started
typed subject
typed date
entered time
Saved
Operation Complete
Operation Complete
Emails finished
Operation Complete
Skipping New Task
Operation Complete
Emails finished
Operation Complete
Task Started
typed subject
typed date
entered time
Saved
Operation Complete
Skipping Status
Emails finished
Operation Complete
Task Started
typed subject
typed date
entered time
Saved
Operation Complete
Operation Complete
Skipping Email
Emails finished
Operation Complete
Task Started
typed subject
typed date
entered time
Saved
Operation Complete
Operation Complete
Skippi

In [84]:
myrange = len(newdf['url'])
for x in range(myrange):
    driver.get(newdf['url'][x])
    if newdf['newtask'][x]ga not in newsoup().get_text():
        print(newdf['name'][x])

TypeError: 'in <string>' requires string as left operand, not float

# END Labor Automation Protocols

# Appendix 1: Set Up Google Calendar Appt (Mass Invite)

In [ ]:
#Google has no way to send mass invites to calendar appointments.
#This module allows me to do that. Not critical to workflow, but helpful

#Literally copy and paste a single column of email addresses from a csv/xls file
names = """""".replace('\n','\n\n\n')
#Navigate manually to the desired calendar page before executing this code
driver.switch_to.window(driver.window_handles[0])
#time.sleep for 5 secs to allow operator to prepare
time.sleep(5)
#locate a valid field
box = driver.find_element_by_class_name("T2Ybvb.KRoqRc.editable")
#manually click the invite field; this is the only way to enter data into it using Python;
#the field refuses to respond to selenium attempts at targetting it directly
f = "\n"
box.send_keys(f'1{f*5}2{f*5}3{f*5}4{f*5}5{f*5}6{f*5}7{f*5}8{f*5}9{f*5}10{f*5}@error.com\n{names}')

for myname in names:    
    if myname not in newsoup().get_text():
        print(myname)

# Close Out App

In [ ]:
driver.quit()